In [ ]:
import numpy as np
import torch
import torchvision
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.utils.data.dataset import random_split
from torchvision import models
from torch.utils.data import Dataset
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd '/content/drive/MyDrive/AI_project'

/content/drive/MyDrive/AI_project


In [ ]:
#Clone attack repository
# !git clone https://github.com/Harry24k/adversarial-attacks-pytorch.git
# !cd adversarial-attacks-pytorch/torchattacks
# !pip install -e .
# !mv adversarial-attacks-pytorch attacks

Cloning into 'adversarial-attacks-pytorch'...
remote: Enumerating objects: 3644, done.
remote: Counting objects: 100% (595/595), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 3644 (delta 434), reused 450 (delta 409), pack-reused 3049
Receiving objects: 100% (3644/3644), 51.81 MiB | 9.37 MiB/s, done.
Resolving deltas: 100% (2274/2274), done.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
from attacks.torchattacks import PGD
from Models import *

# 1. Dataset Preparation

In [ ]:
train_transform = transforms.Compose([
        transforms.RandomCrop(32,padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_transform =  transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform = train_transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,

                                       download=True, transform = test_transform)

# 2. Prepare Network

In [ ]:
class DenoiseLoss(nn.Module):
    def __init__(self, n, hard_mining = 0, norm = False):
        super(DenoiseLoss, self).__init__()
        self.n = n
        assert(hard_mining >= 0 and hard_mining <= 1)
        self.hard_mining = hard_mining
        self.norm = norm

    def forward(self, x, y):
        loss = torch.pow(torch.abs(x - y), self.n) / self.n
        if self.hard_mining > 0:
            loss = loss.view(-1)
            k = int(loss.size(0) * self.hard_mining)
            loss, idcs = torch.topk(loss, k)
            y = y.view(-1)[idcs]

        loss = loss.mean()
        if self.norm:
            norm = torch.pow(torch.abs(y), self.n)
            norm = norm.data.mean()
            loss = loss / norm
        return loss

In [ ]:
# %cd 'content/drive'
class Net(nn.Module):
    def __init__(self, pre_res ,n = 2, hard_mining = 0, loss_norm = False):
        super(Net, self).__init__()
        self.denoiser = Denoise() #Denoiser: to be trained
        self.loss = DenoiseLoss(n, hard_mining, loss_norm)
        self.net = pre_res
        for param in self.net.parameters():
            param.requires_grad = False #Freeze the pretrained base model:Resnet in this case

    def forward(self, orig_x, adv_x, requires_control = True, train = True):
        orig_outputs = self.net(orig_x)

        if requires_control:
            control_outputs = self.net(adv_x)
            control_loss = self.loss(control_outputs, orig_outputs)

        if train:
            adv_x.volatile = False
            for i in range(len(orig_outputs)):
                orig_outputs[i].volatile = False
        adv_x = self.denoiser(adv_x) #denoised adv_x
        adv_outputs = self.net(adv_x) #logit result
        loss = self.loss(adv_outputs, orig_outputs) #
        return orig_outputs, adv_outputs, loss

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train(dataloader,net, attack,optimizer): #adv+benign 1:1
    criterion =  nn.CrossEntropyLoss()
    net.train()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        if(batch%20==0):
          print(f'batch no. {batch}')
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        adv_x = attack(X,y)
        orig_outputs,adv_outputs,loss = net(X, adv_x, requires_control = True, train = True)
        loss = loss+criterion(adv_outputs,y) #clean 과 denoised의 logit간 차이+denoised CE loss

        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (orig_outputs.argmax(1) == y).type(torch.float).sum().item()
        train_acc += (adv_outputs.argmax(1) == y).type(torch.float).sum().item()

    train_loss /= num_batches
    train_acc /= (2*size)
    print(f"HGD Train: \n Accuracy: {(100*train_acc):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    return train_loss, train_acc

def validation_loop(dataloader, net, attack):
    loss_fn = nn.CrossEntropyLoss()
    net.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    total_loss,total_acc = 0, 0
    benign_loss, benign_acc = 0, 0
    adv_loss, adv_acc = 0, 0
    for batch,(X,y) in enumerate(dataloader):
      with torch.no_grad():
        X= X.to(device)
        y = y.to(device)
        _,denoised_clean,_ = net(X, X, requires_control = False, train = False)
        benign_loss += loss_fn(denoised_clean, y).item()
        benign_acc += (denoised_clean.argmax(1) == y).type(torch.float).sum().item()
      X.requires_grad = True
      adv = attack(X,y).to(device)
      X.requires_grad = False
      with torch.no_grad():
        _,denoised_adv,_ = net(X, adv, requires_control = False, train = False)
        adv_loss += loss_fn(denoised_adv, y).item()
        adv_acc += (denoised_adv.argmax(1) == y).type(torch.float).sum().item()

    # Compute total loss and accuracy
    benign_loss /= num_batches
    benign_acc /= size
    adv_loss /= num_batches
    adv_acc /= size
    total_loss = (benign_loss + adv_loss) / 2
    total_acc = (benign_acc + adv_acc) / 2
    print('\nTotal benign test accuarcy:', 100.*benign_acc)
    print('Total adversarial test Accuarcy:', 100.*adv_acc)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)
    print('**Summary**')
    print('Total validation accuracy:',100*total_acc)
    print('Total validiation loss:',total_loss)

    return total_loss,total_acc,benign_loss,benign_acc,adv_loss,adv_acc

In [ ]:
def run(net,optimizer,model_save_name,epochs):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

   # Move model to device
    net = net.to(device)
    attack = PGD(net.net, eps=8/255,alpha=2/255,steps=7)
    best_validation_acc =  29.86
    for t in range(epochs):
        print(f"Epoch {t+6}\n-------------------------------")
        train_loss, train_acc = train(trainloader, net, attack, optimizer)
        total_loss,total_acc,benign_loss,benign_acc,adv_loss,adv_acc = validation_loop(valloader, net, attack)

        if total_acc > best_validation_acc:
            best_validation_acc = total_acc
            print(f"Net at epoch {t+6} saved")
            path = F"/content/drive/MyDrive/AI_project/SaveModel/best_{model_save_name}"
            torch.save(net.state_dict(), path)
        path = F"/content/drive/MyDrive/AI_project/SaveModel/last_{model_save_name}"
        torch.save(net.state_dict(), path)

    print("Done!")

In [ ]:
batch_size = 64
learning_rate=1e-3

In [ ]:
torch.manual_seed(42)
train_len = len(trainset)
num_train = int(train_len * 0.8)
num_val = train_len - num_train
train_set,val_set = random_split(trainset,[num_train, num_val])

trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                              shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                            shuffle=False, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

In [ ]:
pre_res = ResNet50()
PATH = '/content/drive/MyDrive/AI_project/SaveModel/best_plain_res.pt'
pre_res.load_state_dict(torch.load(PATH))

torch.manual_seed(42)
PATH = '/content/drive/MyDrive/AI_project/SaveModel/best_HGDnet_preplain.pt'
net = Net(pre_res,n=2)
net.load_state_dict(torch.load(PATH))
optimizer = optim.Adam(net.parameters(), lr=learning_rate,weight_decay=1e-4)
run(net,optimizer,model_save_name="HGDnet_preplain.pt",epochs = 5)

In [ ]:
#Testing on black box attack
def test_loop(dataloader, net, attack):
    loss_fn = nn.CrossEntropyLoss()
    net.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    total_loss,total_acc = 0, 0
    benign_loss, benign_acc = 0, 0
    adv_loss, adv_acc = 0, 0
    for batch,(X,y) in enumerate(dataloader):
      with torch.no_grad():
        X= X.to(device)
        y = y.to(device)
        _,denoised_clean,_ = net(X, X, requires_control = False, train = False)
        benign_loss += loss_fn(denoised_clean, y).item()
        benign_acc += (denoised_clean.argmax(1) == y).type(torch.float).sum().item()
      X.requires_grad = True
      adv = attack(X,y).to(device)
      X.requires_grad = False
      with torch.no_grad():
        _,denoised_adv,_ = net(X, adv, requires_control = False, train = False)
        adv_loss += loss_fn(denoised_adv, y).item()
        adv_acc += (denoised_adv.argmax(1) == y).type(torch.float).sum().item()

    # Compute total loss and accuracy
    benign_loss /= num_batches
    benign_acc /= size
    adv_loss /= num_batches
    adv_acc /= size
    total_loss = (benign_loss + adv_loss) / 2
    total_acc = (benign_acc + adv_acc) / 2
    print('\nTotal benign test accuarcy:', 100.*benign_acc)
    print('Total adversarial test Accuarcy:', 100.*adv_acc)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)
    print('**Summary**')
    print('Total Test accuracy:',100*total_acc)
    print('Total validation:',total_loss)

    return total_loss,total_acc,benign_loss,benign_acc,adv_loss,adv_acc

In [ ]:
PATH = '/content/drive/MyDrive/AI_project/SaveModel/best_HGDnet_preplain.pt'
loaded_model = Net()
loaded_model.load_state_dict(torch.load(PATH))
loaded_model.to(device)
loaded_model.eval()
attack = PGD(loaded_model.net, eps=8/255,alpha=2/255,steps=7)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False,num_workers=2)
total_loss,total_acc,benign_loss,benign_acc,adv_loss,adv_acc = test_loop(testloader,loaded_model,attack)